In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical  
# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
save_check_pt = './checkpoints/weather'
my_epoch = 1

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = cv2.imread(img_dir)
#         img = cv2.resize(img,(32,32))
#         X.append(img)
#     X = np.array(X)
#     return X

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L') # conver to grayscale images
        img = img.resize([32, 16])
        img_np = np.asarray(img)
        X.append(img_np.reshape([-1]))
    X = np.array(X)
    return X


def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    #model.compile(optimizer=Adam(0.0001), loss='mse')
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(35, activation="sigmoid"))
    #model.compile(optimizer=Adam(0.0001), loss='mse')
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #model.compile(optimizer=Adam(0.0001), loss='CategoricalCrossentropy')
    return model

base_images_dir, base_labels, base_table = load_ccpd(dataset_path['base'],mode='all')
base_images = readCcpdImg(base_images_dir)
base_labels = np.array(base_labels)
base_labels = to_categorical(base_labels, num_classes=35)
weather_images_dir, weather_labels, weather_table = load_ccpd(dataset_path['weather'],mode='all')
weather_images = readCcpdImg(weather_images_dir)
weather_labels = np.array(weather_labels)
weather_labels = to_categorical(weather_labels, num_classes=35)
weather_images_dir_t, weather_labels_t, weather_table_t = load_ccpd(dataset_path['weather'],mode='train')
weather_images_t = readCcpdImg(weather_images_dir_t)
weather_labels_t = np.array(weather_labels_t)
weather_labels_t = to_categorical(weather_labels_t, num_classes=35)
print(np.shape(base_images))
print(np.shape(weather_images))
print(np.shape(weather_images_t))
model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
model.fit(base_images, base_labels,weather_images_t,weather_labels_t, epochs=500,verbose=1)
model.summary()
model.score(base_images, base_labels)
model.score(weather_images, weather_labels)# loss: 4.5758 - acc: 0.7671
model.save_weights(save_check_pt)

(8446, 512)
(3812, 512)
(762, 512)
Epoch 1/500
286/286 [==============================] - 2s 3ms/step - loss: 3.2184 - acc: 0.2222 - disc_loss: 1.3938 - disc_acc: 0.4590
Epoch 2/500
286/286 [==============================] - 1s 4ms/step - loss: 2.7675 - acc: 0.2531 - disc_loss: 1.3887 - disc_acc: 0.4708
Epoch 3/500
286/286 [==============================] - 1s 4ms/step - loss: 2.2701 - acc: 0.3702 - disc_loss: 1.3832 - disc_acc: 0.5241
Epoch 4/500
286/286 [==============================] - 1s 3ms/step - loss: 1.8082 - acc: 0.5271 - disc_loss: 1.3792 - disc_acc: 0.5373
Epoch 5/500
286/286 [==============================] - 1s 3ms/step - loss: 1.5139 - acc: 0.6132 - disc_loss: 1.3797 - disc_acc: 0.5335
Epoch 6/500
286/286 [==============================] - 1s 4ms/step - loss: 1.3019 - acc: 0.6695 - disc_loss: 1.3797 - disc_acc: 0.5324
Epoch 7/500
286/286 [==============================] - 1s 4ms/step - loss: 1.1367 - acc: 0.7030 - disc_loss: 1.3795 - disc_acc: 0.5311
Epoch 8/500
286/286 

286/286 [==============================] - 1s 3ms/step - loss: 0.0596 - acc: 0.9846 - disc_loss: 1.3812 - disc_acc: 0.5285
Epoch 62/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0428 - acc: 0.9922 - disc_loss: 1.3832 - disc_acc: 0.5252
Epoch 63/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0496 - acc: 0.9892 - disc_loss: 1.3826 - disc_acc: 0.5200
Epoch 64/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0754 - acc: 0.9804 - disc_loss: 1.3820 - disc_acc: 0.5243
Epoch 65/500
286/286 [==============================] - 1s 3ms/step - loss: 0.1288 - acc: 0.9630 - disc_loss: 1.3788 - disc_acc: 0.5330
Epoch 66/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0765 - acc: 0.9803 - disc_loss: 1.3797 - disc_acc: 0.5332
Epoch 67/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0542 - acc: 0.9876 - disc_loss: 1.3807 - disc_acc: 0.5307
Epoch 68/500
286/286 [==============================] - 1s 3m

286/286 [==============================] - 1s 3ms/step - loss: 0.0437 - acc: 0.9888 - disc_loss: 1.3837 - disc_acc: 0.5258
Epoch 122/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0839 - acc: 0.9752 - disc_loss: 1.3814 - disc_acc: 0.5290
Epoch 123/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0629 - acc: 0.9826 - disc_loss: 1.3830 - disc_acc: 0.5268
Epoch 124/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0410 - acc: 0.9895 - disc_loss: 1.3828 - disc_acc: 0.5235
Epoch 125/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0293 - acc: 0.9935 - disc_loss: 1.3839 - disc_acc: 0.5245
Epoch 126/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0255 - acc: 0.9942 - disc_loss: 1.3832 - disc_acc: 0.5278
Epoch 127/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0243 - acc: 0.9956 - disc_loss: 1.3841 - disc_acc: 0.5252
Epoch 128/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0327 - acc: 0.9920 - disc_loss: 1.3827 - disc_acc: 0.5297
Epoch 181/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0794 - acc: 0.9773 - disc_loss: 1.3796 - disc_acc: 0.5316
Epoch 182/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0470 - acc: 0.9867 - disc_loss: 1.3805 - disc_acc: 0.5356
Epoch 183/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0324 - acc: 0.9923 - disc_loss: 1.3813 - disc_acc: 0.5301
Epoch 184/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0358 - acc: 0.9906 - disc_loss: 1.3828 - disc_acc: 0.5281
Epoch 185/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0388 - acc: 0.9897 - disc_loss: 1.3802 - disc_acc: 0.5302
Epoch 186/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0286 - acc: 0.9933 - disc_loss: 1.3822 - disc_acc: 0.5242
Epoch 187/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0271 - acc: 0.9941 - disc_loss: 1.3832 - disc_acc: 0.5221
Epoch 240/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0528 - acc: 0.9855 - disc_loss: 1.3811 - disc_acc: 0.5287
Epoch 241/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0679 - acc: 0.9802 - disc_loss: 1.3773 - disc_acc: 0.5410
Epoch 242/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0568 - acc: 0.9846 - disc_loss: 1.3794 - disc_acc: 0.5265
Epoch 243/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0438 - acc: 0.9876 - disc_loss: 1.3801 - disc_acc: 0.5357
Epoch 244/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0292 - acc: 0.9929 - disc_loss: 1.3819 - disc_acc: 0.5285
Epoch 245/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0320 - acc: 0.9928 - disc_loss: 1.3802 - disc_acc: 0.5357
Epoch 246/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0315 - acc: 0.9917 - disc_loss: 1.3833 - disc_acc: 0.5245
Epoch 299/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0364 - acc: 0.9906 - disc_loss: 1.3824 - disc_acc: 0.5277
Epoch 300/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0243 - acc: 0.9942 - disc_loss: 1.3818 - disc_acc: 0.5323
Epoch 301/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0204 - acc: 0.9957 - disc_loss: 1.3834 - disc_acc: 0.5264
Epoch 302/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0249 - acc: 0.9943 - disc_loss: 1.3836 - disc_acc: 0.5206
Epoch 303/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0285 - acc: 0.9931 - disc_loss: 1.3850 - disc_acc: 0.5118
Epoch 304/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0519 - acc: 0.9851 - disc_loss: 1.3820 - disc_acc: 0.5287
Epoch 305/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0365 - acc: 0.9899 - disc_loss: 1.3800 - disc_acc: 0.5339
Epoch 358/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0228 - acc: 0.9950 - disc_loss: 1.3825 - disc_acc: 0.5324
Epoch 359/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0224 - acc: 0.9953 - disc_loss: 1.3827 - disc_acc: 0.5297
Epoch 360/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0190 - acc: 0.9967 - disc_loss: 1.3836 - disc_acc: 0.5289
Epoch 361/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0283 - acc: 0.9937 - disc_loss: 1.3842 - disc_acc: 0.5267
Epoch 362/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0594 - acc: 0.9839 - disc_loss: 1.3833 - disc_acc: 0.5268
Epoch 363/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0502 - acc: 0.9861 - disc_loss: 1.3813 - disc_acc: 0.5335
Epoch 364/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0216 - acc: 0.9952 - disc_loss: 1.3848 - disc_acc: 0.5169
Epoch 417/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0479 - acc: 0.9867 - disc_loss: 1.3824 - disc_acc: 0.5297
Epoch 418/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0327 - acc: 0.9907 - disc_loss: 1.3826 - disc_acc: 0.5264
Epoch 419/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0422 - acc: 0.9890 - disc_loss: 1.3828 - disc_acc: 0.5265
Epoch 420/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0314 - acc: 0.9929 - disc_loss: 1.3843 - disc_acc: 0.5175
Epoch 421/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0180 - acc: 0.9968 - disc_loss: 1.3835 - disc_acc: 0.5204
Epoch 422/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0231 - acc: 0.9952 - disc_loss: 1.3834 - disc_acc: 0.5245
Epoch 423/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0235 - acc: 0.9951 - disc_loss: 1.3834 - disc_acc: 0.5253
Epoch 476/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0176 - acc: 0.9972 - disc_loss: 1.3855 - disc_acc: 0.5229
Epoch 477/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0191 - acc: 0.9964 - disc_loss: 1.3855 - disc_acc: 0.5201
Epoch 478/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0166 - acc: 0.9964 - disc_loss: 1.3855 - disc_acc: 0.5132
Epoch 479/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0172 - acc: 0.9964 - disc_loss: 1.3836 - disc_acc: 0.5224
Epoch 480/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0513 - acc: 0.9865 - disc_loss: 1.3842 - disc_acc: 0.5190
Epoch 481/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0503 - acc: 0.9855 - disc_loss: 1.3805 - disc_acc: 0.5321
Epoch 482/500
286/286 [==============================] 

In [2]:
# fig = plt.figure()
# img_h, img_w = 32,16
# fig.subplots_adjust(hspace=0.4, wspace=0.4)
# for i in range(6):
#     ax = fig.add_subplot(2, 3, i+1)
#     ax.imshow(weather_images[i].reshape([img_h, img_w]))
#model.save_weights('ckpt')
#dummy step to load model from checkpoint
#xt = np.zeros((3812, 512))
#nmd = DANN(lambda_=0.1, Xt=xt, metrics=["acc"], random_state=0)
#x = np.zeros((1,1))
#y = np.zeros(1)
#nmd.fit(x,y, epochs=0,verbose=1)
#nmd.summary()
#nmd.load_weights("ckpt")
#nmd.score(weather_images, weather_labels)
print(model.predict(weather_images[0].reshape([-1,512])))

[[1.37786021e-19 3.39480223e-20 9.74124766e-26 1.04812023e-20
  3.18515735e-28 9.10637191e-12 6.99350692e-27 8.70688741e-18
  9.88755675e-11 1.54152890e-09 7.38655352e-36 7.94960819e-10
  0.00000000e+00 5.95084072e-20 2.33703197e-28 2.92184951e-21
  3.60168527e-24 6.22940055e-09 1.77201020e-27 5.49169325e-13
  1.09686239e-21 0.00000000e+00 1.01760024e-10 2.76643842e-01
  2.40978910e-23 5.47434810e-15 2.21299853e-10 1.66838918e-10
  7.00208554e-13 2.32378275e-16 2.06933026e-10 5.98121812e-14
  4.26281536e-12 7.45897023e-06 8.93831931e-19]]
